## Chapter 8: Water treatment

[Video on Aerobic Digestion](https://youtu.be/zoeBqgp86rY)

By now you should have an idea of the food chain, how it is produced, how it is consumed and how waste from the process can be used to generate energy. This course has a biochemical engineering focus and accordingly we get a bit deeper into how micro-organisms or microbes are part of the food chain. As chemical engineers it is imperative to understand how microbes can be utilised to process product streams from the food chain. In the previous two chapters we have considered the processing of manure, foodwaste and non-edible plant material. It is important to understand that a major fraction of the nutrients from the food chain ends up in water, just think of what you flush down daily in the toilet. In this regard it is important to understand how microbes can assist in cleaning the water. Have a look at a generic outlay of a wastewater plant:

<img src="water1.png" width="650" />

You will see that the intial steps of wastewater treatment entails the removal of solids from the wastewater and that the collected solids are fed to an anaerobic digestion unit in order to generate energy. Once solids are removed there is still a lot of dissolved organics in the water and microbes are required to process these chemicals. Our focus in this chapter will mainly be on the biological treatment of the solid-lean water, where aerobic organisms are used to digest the nutrients. We can talk about aerobic digestion and approach the modeling in a very similar manner than before. The main difference will be the aeration required to supply the organisms with oxygen. To purify the water, carbon, nitrogen and minerals (especially phosphorous) have to be removed. Since the process is aerobic carbon can be metabolised to $CO_2$ that will escape as a gas. Nitrogen still has to be released from its organic state to form $NH_3$. There will also be a lot of $NH_3$ in the water from urine. In standard wastewater terminlogy the term **secondary wastewater treatment** is used to describe the biological processes. These processes can occur in multiple vessels and apart from removing carbon to $CO_2$, nitrogen is typically removed as $N_2$. The processes of converting $NH_3$ to $N_2$ is referred to as nitrification and denitrification, the latter being an anaerobic process. Note that minerals will be removed via live microbes referred to as **sludge**. Sludge can be fed to anaerobic digesters to decrease the carbon content and to free the nitrogen and minerals contained within the sludge.

In this chapter we will only look at carbon removal and $NH_3$ production from dissolved organic matter. We will work exclusively with continous processes since this is the standard mode of operation. Before we get into the modeling we'll look at standard measuring techniques to quantify the quality of wastewater. 

## COD and TOC

Chemical oxygen demand (COD) and total organic carbon (TOC) are standard measurement techniques in wastewater treatment. It is easy to understand these measurements by considering the mass balance where a given dissolved organic substance ($CH_xO_yN_z$) are fully oxidised to $CO_2$ and $N_2$. Let's consider the following reaction:

$$S+O_2 \rightarrow CO_2+N_2+H_2O$$

<img src="mat1.png" width="550" />


Note that the single specification states that one cmol of organic carbon is oxidised. COD is typically reported in milligrams of oxygen per liter while TOC is given in milligrams of carbon. Let's calculate:

In [4]:
import pandas
import numpy as np
S = np.matrix(pandas.read_excel('mat1a.xlsx'))

C= np.matrix([0, 0, 0,0,-1]).T
r = np.linalg.solve(S, C)
r

matrix([[-1.   ],
        [-1.075],
        [ 1.   ],
        [ 0.055],
        [ 0.9  ]])

In [5]:
MMs=12+S[1,0]+S[2,0]*16+S[3,0]*14
COD=-r[1]*32 #gram of oxygen
TOC=-r[0]*12 #gram of carbon
[float(COD), float(TOC), float(COD/TOC)]

[34.4, 11.999999999999998, 2.866666666666667]

Depending on the concentration of the organics in the water we can express the COD or TOC as an concentration. Typical COD values in wastewater range from 500 to 50000 $\frac{mg}{L}$. The COD to TOC typically varies between 1.5 and 4. Note that both TOC and COD gives an indication of the organic content of water, while COD is dependant on the oxidation state of the organic substance. 

For the example above we converted 1 cmol of substrate that requires 34.4 grams of oxygen. If this 1 cmol was in 1 liter of water the COD would be 34400 $\frac{mg}{L}$.

## Aerobic digestion of wastewater

The biological breakdown of organics is facilitated by microbes. The first step is to convert the carbon to microbial cells and $CO_2$ and the nitrogen to microbial cells and $NH_3$. The following reaction describes the overall reaction: 


$$S+O_2 \rightarrow X+CO_2+NH_3+H_2O$$


Let's import the matrix:

<img src="mat2.png" width="550" />


In [6]:
S1 = np.matrix(pandas.read_excel('mat1.xlsx'))
S1

matrix([[ 1.  ,  0.  ,  1.  ,  1.  ,  0.  ,  0.  ],
        [ 1.8 ,  0.  ,  1.6 ,  0.  ,  2.  ,  3.  ],
        [ 0.75,  2.  ,  0.75,  2.  ,  1.  ,  0.  ],
        [ 0.11,  0.  ,  0.08,  0.  ,  0.  ,  1.  ],
        [ 0.  , -2.5 , -2.5 ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ]])

Looking at the fifth row we see the energy balance. You will see that the $\gamma$ value of 2.5 $\frac{mol \, ATP}{cmol \, X }$ is higher than the anaerobic value of 1.8 $\frac{mol \, ATP}{cmol \, X }$. This is becuase ATP is far more avaialble in a aerobic system and accordingly the organism is less efficient in conserving ATP. Remember from chapter 1 that we got 21 ATP from a mole of glucose, this will translate to 3.5 mol ATP per cmol of glucose. Compare this to the amount of ATP obtained in lactic acid (anaerobic) fermentation where only 2 ATPs are obtained per mole of glucose (or $\frac{1}{3}$ mol ATP per cmol of glucose). In this example we will assume that the breakdown of the organic to pyruvic acid will entail the use of ATP and accordingly we'll generate 2.5 mole of ATP per oxygen consumed. Note that we use the oxygen stream to assign the ATP generation rather than the substrate. This makes sense since it is easier to relate ATP generation with oxygen (remember the P/O ratios in chapter 1).

Note that the $\mu$ and $\theta$ values are higher than before. These are typical values for aerobic digestion. Aerobic digestion is also referred to as the activated sludge process, where the activated sludge refers to the microbial biomass in the process.  

### Air sparging

Before we go further we need to understand how oxygen is transferred into the reactor liquid. Sparging entails bubbling of air through liquid. The objective is to maximise gas-liquid interfacial area (with small bubbles) while maintaining good agitation in the digester.

<img src="sparger.jpg" width="450" />


Let's have a look at describing gas-liquid mass transfer. The mathematical formulation of the volumetric (reactor based) mass transfer rate is given by:

$$r_{O}^{mt}=k_La \cdot \left(C_{O}^{sat}-C_{O} \right)$$
--------
$$ \left[ \frac{mol\,O}{L\cdot day}  \right] = \left[ \frac{1}{day} \cdot \frac{mol\,O}{L}  \right]$$

Note that $r_{O}^{mt}$ is a volume based rate. The rate is defined by a mass transfer coefficient ($k_La$) multiplied by a concentration difference between the saturated oxygen concentration ($C_{O}^{sat}$) and the oxygen concentration in the fermenter ($C_{O}$). The saturated oxygen concentration is dependent on the oxygen partial pressure and typically varies between $6-8 \frac{mg}{L}$ (1.9e-4 to 2.5e-4 $\frac{mol\,O_2}{L}$) at atmospheric air conditions The chemical engineering approach is to assume that all liquid on the gas-liquid interface (edge of bubbles) are saturated (at $C_{O}^{sat}$ ) while $C_{O}$ will always be less than $C_{O}^{sat}$. The mass transfer coefficient (note the units) is determined by the air flowrate, sparger and impeller characteristics but will be supplied in this course.

We will start our modeling with a batch system. In this system the microbial biomass will grow and $C_X$ will increase. With an increase in $C_X$ the demand for oxygen will increase since both energy drivers ($\theta$ and $\mu$) derives energy from using energy. A situation will develop where the amount of oxygen that can be supplied via mass transfer ($kla(C_o^{sat}-C_o)$) will determine the amount of growth and maintenance that cells can obtain. Each cell requires a minimum maintenance amount to sustain itself and this value tends to be close to $\theta_{max}$. If $\theta$ drops below this amount the cells in an oxygenated system struggle to maintain themself. To model this requirement we first need to look at the governing rate equations:

$$\mu=\mu_{max}(\frac{C_S}{K_S^{\mu}+C_S})(\frac{C_O}{K_O^{\mu}+C_0})$$

$$\theta=\theta_{max}(\frac{C_S}{K_S^{\theta}+C_S})(\frac{C_O}{K_O^{\theta}+C_0})$$

We see from these equations that oxygen behaves very similar to the biomass substrate ($S$) and that low concentrations of oxygen ($C_O$) will slow down both $\theta$ and $\mu$. Since $|theta$ has to stay close to the maximum value we will use a $K_O^{\theta}$ value that is very small. This implies that even under very low $C_O$ conditions $\theta$ will be close to its maximum value. Let's assume that biomass substrate is in excess (Monod term of $C_S$ negelcted) and look at the following two equations:

$$\mu=\mu_{max}(\frac{C_O}{K_O^{\mu}+C_0})$$

$$\theta=\theta_{max}(\frac{C_O}{K_O^{\theta}+C_0})$$

a) Given the $K_O$ values below you should get the following graph:


In [10]:
MMx=12+S1[1,2]+S1[2,2]*16+S1[3,2]*14
MMs=12+S[1,0]+S[2,0]*16+S[3,0]*14
thetamax=0.5/1000*MMx*24
#0.5 molATP/g/h
mumax=0.1
# 1/day
Co_sat=7/1000/32 
#7mg/L is standard for maximum amount of oxygen that can dissolve. This is the saturation concentration of CS
Ko_mu=Co_sat*0.15 #mol/L
Ko_theta=Co_sat*0.0000001 

You should get the following:

<img src="monod.png" width="550" />

Note that $\mu$ will never be able to operate at $\mu_{max}$ unless pure oxygen is sparged. Note that $\mu$ decreases rapidly at lower $C_O$ while $\theta$ remains at the maximum value, even at low concentrations of $C_O$. This imply that low oxygen concentrations in the reactor will cause cells to spend the majority of energy on maintenance and little energy on growth. 

### Model a batch aerobic digester with activated sludge

Let's now model a batch aerobic digester. The following information is given:

In [8]:
Ks_mu=0.0007 #cmol/L
Ks_theta=0.007/1e6
kla=20  #per day
Cso=20/MMs   #cmol/L
Cxo=0.03/MMx  #cmol/L
Coo=0.5*Co_sat

b) Set up your own response function. Remeber that the oxygen balance will reduce to:

$$\frac{dC_O}{dt}= r_O C_X+k_La \cdot \left(C_{O_2}^{sat}-C_{O_2} \right)$$

Try to obtain the following graphs:

<img src="8b1.png" width="550" />
<img src="8b2.png" width="550" />

Note that the growth appears to terminate while biomass substrate ($C_S$) is still available. It is evident that the $C_O$ drops significantly as $C_X$ in the reactor increases. This implies that maintenance will dominate the energy expenditure towards the end of the run.

c) Let's plot $\mu \gamma$ and $\theta$ to determine the ratio of energy expenditure within the cells. You should get the following:

<img src="8c.png" width="550" />


d) What happens if the kla value is doubled to 40 $\frac{1}{day}$. Plot the substrate profiles for both runs, you should get this:

<img src="8d1.png" width="550" />
<img src="8d2.png" width="550" />

Explain the reason for the difference!

## Continuous aerobic digestion 

We are now ready to see how a continous digester will perform. Use a dilution rate of $\frac{1}{18}$ $\frac{1}{day}$, a kla of 20 $\frac{1}{day}$, and 2 $\frac{g}{L}$ of substrate in the feedstream. Also use the following initial concentrations in the reactor:  

In [9]:
Cstart=[3/MMs,0.5*Co_sat,0.03/MMx,0] #S,O,X.N

e) Show that the dynamic response of the system looks like the following:

<img src="8e1.png" width="550" />
<img src="8e2.png" width="550" />


f) Determine the reduction in COD for the run in (e). [2516 to 109 $\frac{mg}{L}$]

g) If the kla value is reduced to 5 $\frac{1}{day}$, determine the decrease in substrate conversion.
[95.6% to 28.15%]

What is the steady state oxygen saturation of run (e) and run (g)? 
[31.7% to 19.3%]

What is the $C_X$ of run (e) and run (g)? 
[0.434 to 0.128 $\frac{g}{L}$]

Can you now understand how important proper aeration is in an aerobic digester?



h) If you keep the kla at 5 $\frac{1}{day}$, determine the hydraulic retention time to achieve the same substrate conversion as in (e). How much bigger should the reactor be if the feed flowrate (Q) to (e) and (h) remains the same. [3.7 times]. So when considering the cost of the reactor we will have to balance the cost of the size of the reactor against the cost of the mass transfer equipment. Think about this.